#### PIPE - 6:    
This pipe covers the following : 
1. All features of pipe 5
2. Blurring rest image (mild) while saving it for evidance in evidance folder Keep both images
3. Selecting images based on size for enhancement to save time
4. Detecting Image using YOLO model two (Part one of Phase one ANRP)
5. Cropping liscense plate images for OCR purpose (Part one of Phase one ANRP)

In [1]:
%%time
import cv2
import numpy as np
import pandas as pd
import torch
import time
import threading 
from ultralytics import YOLO
from pathlib import Path
from boxmot import StrongSORT
from utils.utils import *
start_time=time.time()
down = {}
up = {}
text_color = (0, 0, 0)  # Black color for text
yellow_color = (0, 255, 255)  # Yellow color for background
red_color = (0, 0, 255)  # Red color for lines
blue_color = (255, 0, 0)  # Blue color for lines

counter_down = []
counter_up = []

first_boundry_y=170

red_line_y = 198

blue_line_y = 280

offset = 6
# Specify the start and end points of the line
start_point = (300, 198)
end_point = (300, 280)

all_id_tracker=[]
voilation_id_tracker=[]
violation_frame_tracker={}
tracker=tracker_init(cuda_device=torch.cuda.is_available(),cuda_device_number=1)
model=YOLO('yolov8s.pt')
video_path='/media/hlink/hd/test_videos/testx_vid_1.mp4'
vid = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out_main = cv2.VideoWriter('output_strongs.avi', fourcc, 20.0, (1020, 500))
frame_count=0
all_frames_record_path=create_directory('all_frames_record')
violation_frames_record_path=create_directory('all_violation_record')

while True:    
    ret, im = vid.read()
    
    if not ret:
        break
    
    im=cv2.resize(im,(1020, 500))
    
    
    results=model.predict(im,conf=0.4,verbose=False,device=[1],classes=[2,3])
    # print(f"\n----------------------------------------------------------------------------------\nResult without detection \n{results[0].boxes.data}\n----------------------------------------------------------------------------------\n")
    im=line_plotter(frame=im,x_start=300,x_end=774,y=198,line_color=red_color,line_name='boundry 1',line_thickness=2,text_color=text_color)
    im=line_plotter(frame=im,x_start=300,x_end=800,y=280,line_color=blue_color,line_name='boundry 2',line_thickness=2,text_color=text_color)
    
    # Specify the color of the line in BGR format (here, it's white)
    color = (255, 255, 255)
    
    # Draw the vertical line on the image
    thickness = 2  # You can adjust the thickness as needed
    cv2.line(im, start_point, end_point, color, thickness)
    
    # im=cv2.rectangle(im, (300, 198), (800, 280), (255, 255, 255), 2)
    
    px,conf=prediction_coordinated_hadler(results)
    dets = []
    # Experimenting
    dets=tracker_element_creator(dets,px,conf)        
    
    # print('out_from')
    
    dets = np.array(dets)
    # print(dets)
    if len(dets) > 0:
        tracks = tracker.update(dets, im) # --> M X (x, y, x, y, id, conf, cls, ind)
        # print('tracks',tracks)    
        for track in tracks:
                im,object_id,cx,cy,bbox_list=plot_tracks(track,im)
                # print("vblkjsenjlks",bbox_list)
                all_id_tracker.append(object_id)
                # Adding file path for csv
                csv_file_path=detection_coordinate_write(frame_count,object_id,bbox_list,'all_frame_detection_detail.csv')
            
                if red_line_y<(cy+offset) and red_line_y > (cy-offset):
                    # print("entered if 1")
                    down[object_id]=time.time()
            
                if object_id in down:
                    # print("entered if 2")
               
                    if blue_line_y<(cy+offset) and blue_line_y > (cy-offset):
                         # print("entered if 3")
                         elapsed_time=time.time() - down[object_id] 
                         # print("entered if 4")
                         if counter_down.count(object_id)==0:
                            counter_down.append(object_id) 
                            distance = 9
                            est_speed=speed_calculator_kmph(distance,elapsed_time)
                            if est_speed>20:
                                print(f'violation_detected : {object_id}')
                                voilation_id_tracker.append(object_id)
                                violation_frame_tracker[object_id]=frame_count
                                # violation_frame_writer_op=frame_writer(frame_count,im,violation_frames_record_path)
                                print("bbox_list-",bbox_list)
                                violation_frame_writer_op=frame_writer(frame_count,im,violation_frames_record_path)
                                violation_frame_writer_op=frame_writer(frame_count,im,violation_frames_record_path,coordiante_blur=True,bbox_list=bbox_list)
                                violation_csv_file_path=detection_coordinate_write(frame_count,object_id,bbox_list,'violation_frame_detection_detail.csv')
                                
                            print(f"for {object_id} speed is -> {est_speed}")
    
    all_frame_writer_op=frame_writer(frame_count,im,all_frames_record_path)    
    frame_count+=1
    out_main.write(im)
    
    # # break on pressing q or spaceq
    cv2.imshow('BoxMOT detection', im)     
    # key = cv2.waitKey(25) & 0xFF
    
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

vid.release()
cv2.destroyAllWindows()
remove_model_from_gpu(model)
end_time=time.time()
print(f'total_time {end_time-start_time}')

2024-05-17 13:51:21.320 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:207 - Successfully loaded pretrained weights from "models/osnet_x0_25_msmt17.pt"
/home/hlink/workspace/learning/boxmot/bmt/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with an OutOfMemoryError: CUDA out of memory. Tried to allocate 128.00 MiB. GPU  (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:924.)
  return F.conv2d(input, weight, bias, self.stride,


violation_detected : 5
bbox_list- [484, 237, 581, 319]
for 5 speed is -> 32
violation_detected : 10
bbox_list- [463, 223, 570, 329]
for 10 speed is -> 38
violation_detected : 28
bbox_list- [558, 250, 598, 305]
for 28 speed is -> 33
violation_detected : 34
bbox_list- [476, 256, 500, 301]
for 34 speed is -> 37
violation_detected : 72
bbox_list- [369, 230, 464, 326]
for 72 speed is -> 34
violation_detected : 88
bbox_list- [626, 256, 653, 301]
for 88 speed is -> 25
violation_detected : 107
bbox_list- [356, 222, 452, 329]
for 107 speed is -> 38
violation_detected : 113
bbox_list- [418, 221, 523, 336]
for 113 speed is -> 37
violation_detected : 174
bbox_list- [345, 238, 435, 322]
for 174 speed is -> 24
for 192 speed is -> 17
violation_detected : 169
bbox_list- [465, 239, 558, 319]
for 169 speed is -> 24
violation_detected : 180
bbox_list- [461, 237, 545, 316]
for 180 speed is -> 33
violation_detected : 213
bbox_list- [417, 228, 517, 324]
for 213 speed is -> 39
violation_detected : 214
bbox_l

In [2]:
%%time
main_violation_json=voilation_capture_json_creator(voilation_id_tracker,csv_file_path)
maintained=json_frame_order_checker(main_violation_json)
if maintained:
    evidance_img_dir_paths,evidance_clip_dir_paths=evidance_directories_creator(main_violation_json)
    evidance_img_dir_paths=evidance_img_separator(evidance_img_dir_paths,all_frames_record_path,main_violation_json)
    video_clip_creator_mt(voilation_id_tracker,violation_frame_tracker,evidance_clip_dir_paths,all_frames_record_path)
    cropped_images_path_list=evidance_cropper_mt(evidance_img_dir_paths,main_violation_json)
    deblurred_image_paths=deblur_images(cropped_images_path_list,main_violation_json)

NameError: name 'voilation_capture_json_creator' is not defined

In [4]:
%%time
# Using default 8 threads to achive max effiencicy
enhance_image_path=image_enhancement_using_limit_mpx(deblurred_image_paths,num_threads=8)

CPU times: user 3.55 s, sys: 4.06 s, total: 7.61 s
Wall time: 54min 55s


In [ ]:
%%time
preprocssing_part_one(enhance_image_path,'models/LPD.pt',2)

##### ANRP Phase ONE

In [40]:
from ultralytics import YOLO 
modelLPD=YOLO('models/LPD.pt')

detections = modelLPD.predict('s.jpg',save=True,verbose=False,conf=0.40,device=[1])
print(detections)

Results saved to /home/hlink/workspace/anrp/runs/detect/predict5
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'liscense_plate'}
obb: None
orig_img: array([[[42, 38, 33],
        [42, 38, 33],
        [48, 45, 41],
        ...,
        [23, 27, 28],
        [26, 30, 31],
        [36, 40, 41]],

       [[45, 41, 36],
        [43, 39, 34],
        [50, 47, 43],
        ...,
        [19, 23, 24],
        [25, 29, 30],
        [33, 37, 38]],

       [[53, 49, 44],
        [49, 45, 40],
        [58, 54, 49],
        ...,
        [26, 30, 31],
        [25, 29, 30],
        [23, 27, 28]],

       ...,

       [[41, 46, 45],
        [43, 48, 47],
        [42, 47, 46],
        ...,
        [38, 43, 42],
        [35, 40, 39],
        [33, 38, 37]],

       [[15, 17, 17],
        [15, 20, 19],
        [15, 17, 17],
        ...,
        [12, 17, 16],
        [12, 14, 14],
        [ 8, 13, 12]],

  

In [ ]:

import cv2
import numpy as np
from ultralytics import YOLO
import time
import os

def anrp(image_path,yolo_model=None,img_enhancer_model_path=None,tr_model=None,tr_device=None,tr_processor=None):
    # taking model
    model=yolo_model
    
    # image_name=os.path.split(image_path)
    # image_count=os.path.splitext(image_name[0])
    
    # Reading images
    img = cv2.imread(image_path)

    detections = model(img,save=False,verbose=False,conf=0.20)
    
    
    x1, y1, x2, y2,detections_=finding_coordinates(detections)

    if x1!=-1 and y1!=-1 and x2!=-1 and y2!=-1 and detections!=-1:
            x1,y1,w,h=coordinates_creator(x1,y1,x2,y2)
            # Cropping number Plate image
            cropped_img_number_plate=image_cropper_using_arr(img,x1,y1,w,h)
        
            upscaled_image=enhance_image(cropped_img_number_plate,re_upscale=True,model_path=img_enhancer_model_path, save_output=True)    
            
            # ocr_image=image_thereshold_creator(img=cropped_img_number_plate)
            
            lp_1,lp_2,lp_3=run_ocr(upscaled_image,tr_model,tr_device,tr_processor)
            
            final_string=common_chars(lp_1,lp_2,lp_3)
            
            return final_string,cropped_img_number_plate,upscaled_image,ocr_image,image_name[1],image_count
    else :
        del img
        return None,None,None,None,None,None

In [ ]:
def common_chars(input_1, input_2, input_3):
    final_string = ''
    for i in range(min(len(input_1), len(input_2), len(input_3))):
        char_count = {input_1[i]: 1, input_2[i]: 1, input_3[i]: 1}
        if len(char_count) == 1:
            final_string += input_1[i]  # Append any character since all are the same
        elif len(char_count) == 2 and input_2[i] == input_3[i]:
            final_string += input_2[i]  # Append character present in both inputs 2 and 3
        else:
            sorted_chars = sorted(char_count.keys(), key=lambda x: -char_count[x])
            final_string += sorted_chars[0]  # Append the character that appears in more inputs
    return final_string

In [ ]:

def anrp_processor(anrp_folder_paths,yolo_model,img_enhancer_model_path,tr_device,tr_model,tr_processor):
    anrp_detections=[]
    for i in anrp_folder_paths:
        sorted_files=sort_files_by_creation_time(i['main_folder_path'])
        final_json={}
        np_counter={}
        for j in sorted_files:
            final_string,cropped_img_number_plate,upscaled_image,ocr_image,image_name,image_count = anrp(image_path=j,yolo_model=yolo_model,img_enhancer_model_path=img_enhancer_model_path,tr_device=tr_device,tr_model=tr_model,tr_processor=tr_processor)
            if final_string != None:
                    if final_string in np_counter.keys():
                        np_counter[final_string]+=1
                    else:
                        np_counter[final_string]=0
        if len(np_counter)!=0:    
            liscense_plate,detection_number=find_max_key(np_counter) 
            confidence=(detection_number/len(sorted_files))*100
            final_json['count_id']=i['count_id']
            final_json['number_plate_ocr_value']=liscense_plate
            final_json['confidence']=confidence
            final_json['evidance_number_plate_image']=frame_writer(count=image_count,frame=cropped_img_number_plate,folder_path=i['cropped_folder_path'],resize=False)
            final_json['ocr_evidance_number_plate_image']=frame_writer(count=image_count,frame=ocr_image,folder_path=i['ocr_folder_path'],resize=False)
            final_json['upscaled_evidance_number_plate_image']=frame_writer(count=image_count,frame=upscaled_image,folder_path=i['upscaled_folder_path'],resize=False)
                        
            anrp_detections.append(final_json)            
        
    return anrp_detections


In [ ]:
import torch

# Create an empty tensor with shape (0, 6) on the CUDA device (GPU 0)
tensor = torch.empty((0, 6), device='cuda:0')

In [ ]:
len(tensor)==0

In [ ]:
tensor

In [ ]:
import threading
import cv2
from ultralytics import YOLO
from utils.utils import *

def run_tracker_in_thread(filename, model, file_count):
    """
    This function is designed to run a video file or webcam stream
    concurrently with the YOLOv8 model, utilizing threading.

    - filename: The path to the video file or the webcam/external
    camera source.
    - model: The file path to the YOLOv8 model.
    - file_count: An optional argument to specify the count of the
    file being processed.
    """
    dir_path=create_directory(f'mp_test_{file_count}')
    
    video = cv2.VideoCapture(filename)  # Read the video file
    print(f"Starting for file {file_count}")
    count=0
    while True:
        ret, frame = video.read()  # Read the video frames
        
        # Exit the loop if no more frames in either video
        if not ret:
            break
        # Track objects in frames if available
        results = model.predict(frame,device=[1],verbose=False)
        # res_plotted = results[0].plot()
        # cv2.imshow("Tracking_Stream_"+str(file_count), frame)
        file_path=os.path.join(dir_path,f'{count}.jpg')
        # print(file_path)
        cv2.imwrite(file_path,frame)
        count=count+1
        key = cv2.waitKey(1)
        if key == ord('q'):
            break
    print(f"finished for {file_count}")
    # Release video sources
    video.release()


# Define the video files for tracking
video_file1 = '/media/hlink/hd/test_videos/testx_vid_1.mp4'
video_file2 = '/media/hlink/hd/test_videos/testx_vid_2.mp4'
video_file3 = '/media/hlink/hd/test_videos/testx_vid_3.mp4'
video_file4 = '/media/hlink/hd/test_videos/testx_vid_4.mp4'




# Create and start tracking threads
tracker_thread1 = threading.Thread(target=run_tracker_in_thread, args=(video_file1, model1, 1), daemon=True)
tracker_thread2 = threading.Thread(target=run_tracker_in_thread, args=(video_file2, model2, 2), daemon=True)
tracker_thread3 = threading.Thread(target=run_tracker_in_thread, args=(video_file3, model3, 3), daemon=True)
tracker_thread4 = threading.Thread(target=run_tracker_in_thread, args=(video_file4, model4, 4), daemon=True)
tracker_thread1.start()
tracker_thread2.start()
tracker_thread3.start()
tracker_thread4.start()

# Wait for threads to finish
tracker_thread1.join()
tracker_thread2.join()
tracker_thread3.join()
tracker_thread4.join()

# Clean up and close windows
cv2.destroyAllWindows()

In [1]:
import cv2
help(cv2.resize)

Help on built-in function resize:

resize(...)
    resize(src, dsize[, dst[, fx[, fy[, interpolation]]]]) -> dst
    .   @brief Resizes an image.
    .   
    .   The function resize resizes the image src down to or up to the specified size. Note that the
    .   initial dst type or size are not taken into account. Instead, the size and type are derived from
    .   the `src`,`dsize`,`fx`, and `fy`. If you want to resize src so that it fits the pre-created dst,
    .   you may call the function as follows:
    .   @code
    .       // explicitly specify dsize=dst.size(); fx and fy will be computed from that.
    .       resize(src, dst, dst.size(), 0, 0, interpolation);
    .   @endcode
    .   If you want to decimate the image by factor of 2 in each direction, you can call the function this
    .   way:
    .   @code
    .       // specify fx and fy and let the function compute the destination image size.
    .       resize(src, dst, Size(), 0.5, 0.5, interpolation);
    .   @endcode
